In [1]:
import time
from PIL import Image
import imagehash
import cv2
import numpy as np
from tqdm import tqdm
import moviepy.editor as mp
import os

def extract_frames(video_clip, end_frame=None):
    print('Extracting frames..')
    print('fps', video_clip.fps)
    frames = []
    for i, frame in tqdm(enumerate(video_clip.iter_frames(fps=1))):
        if end_frame is not None and i >= end_frame:
            break
        frames.append(frame)
    print('Extraction finished')
    return frames

def resize_frame(frame, size=(720, 1280)):
    return cv2.resize(frame, size)

def extract_frames_and_resize(diggle_highlight_video, merged_video, end_frame=None, size=(720, 1280)):

    diggle_frames = [resize_frame(frame, size) for frame in tqdm(extract_frames(diggle_highlight_video, end_frame=end_frame))]
    merged_frames = [resize_frame(frame, size) for frame in tqdm(extract_frames(merged_video, end_frame=end_frame))]

    return diggle_frames, merged_frames

In [2]:
diggle_highlight_vid_path = '/hub_data2/taehoonlee/diggle/[1-8화 요약] 고구마 없는 매운맛 사이다 전개로 화제의 중심에 선 〈내 남편과 결혼해줘〉, 1-8화 하이라이트 모음집🔥 #내남편과결혼해줘.mp4' # 'video_n_frames': 214407
merged_vid_path = '/hub_data2/taehoonlee/clips/S01E03/merged_video_S01E03.mkv' #  'video_n_frames': 33289

diggle_highlight_video = mp.VideoFileClip(diggle_highlight_vid_path)
merged_video = mp.VideoFileClip(merged_vid_path)

print(diggle_highlight_video.fps, diggle_highlight_video.duration/8*diggle_highlight_video.fps)
print(merged_video.fps, int(merged_video.duration*merged_video.fps))

29.97002997002997 26800.961538461535
23.976023976023978 37945


In [3]:
# diggle_sub_clip = diggle_highlight_video.subclip(0, 100)
# merged_sub_clip = merged_video.subclip(0, 100)

diggle_frames, merged_frames = extract_frames_and_resize(diggle_highlight_video, merged_video)

Extracting frames..
fps 29.97002997002997


7155it [02:53, 41.18it/s]


Extraction finished


100%|██████████| 7155/7155 [00:02<00:00, 3173.05it/s]


Extracting frames..
fps 23.976023976023978


1583it [04:20,  6.08it/s]


Extraction finished


100%|██████████| 1583/1583 [00:00<00:00, 2476.29it/s]


In [4]:
def frame_to_hash(frame):
    """Convert a frame to perceptual hash."""
    img = Image.fromarray(frame)
    return imagehash.phash(img)

def find_similar_timestamps(original_frames, target_frames, hash_threshold=6):
    start_time = time.time()
    """
    Find timestamps where frames in the original video are similar to frames in the target video.
    """
    overlaps = []
    
    # Precompute hashes for both original and target frames
    print("Computing hashes for original frames...")
    original_hashes = [frame_to_hash(frame) for frame in tqdm(original_frames)]
    
    print("Computing hashes for target frames and finding matches...")
    for i, target_frame in enumerate(tqdm(target_frames)):
        target_hash = frame_to_hash(target_frame)
        
        min_diff = float('inf')
        min_diff_idx = -1
        # find one frame in original hases that is most similar to target hash
        for j, original_hash in enumerate(original_hashes):
            # Check Hamming distance
            diff = original_hash - target_hash
            if diff < min_diff:
                min_diff = diff
                min_diff_idx = j
        
        # Filter based on threshold
        if min_diff <= hash_threshold:
            overlaps.append((i, min_diff_idx))
            
    end_time = time.time()
    print(f'Finding overlaps finished in {end_time - start_time:.2f} seconds')
    return overlaps

In [5]:
# 대사 있는 부분에서 diggle에 있는 부분을 찾아야 하기 때문에.. target video는 merged video로 설정

hash_threshold = 10
overlapping_frames = find_similar_timestamps(diggle_frames, merged_frames, hash_threshold=hash_threshold)
print("Overlapping frames:", overlapping_frames)

Computing hashes for original frames...


100%|██████████| 7155/7155 [00:44<00:00, 159.97it/s]


Computing hashes for target frames and finding matches...


100%|██████████| 1583/1583 [00:42<00:00, 37.08it/s]

Finding overlaps finished in 87.42 seconds
Overlapping frames: [(0, 1716), (23, 4506), (99, 590), (124, 3920), (346, 1917), (347, 1917), (350, 1929), (351, 1929), (365, 1737), (366, 1738), (367, 1740), (368, 1740), (369, 1740), (370, 1743), (371, 1747), (372, 1747), (373, 1747), (409, 522), (410, 522), (411, 522), (415, 522), (431, 522), (432, 522), (433, 522), (628, 1434), (629, 1518), (630, 1490), (631, 1491), (633, 1518), (634, 1494), (635, 1497), (636, 1498), (683, 1755), (684, 1755), (685, 1757), (686, 1757), (687, 1759), (688, 1759), (689, 1760), (690, 1762), (691, 1763), (692, 1764), (693, 1765), (694, 1766), (695, 1766), (696, 1768), (697, 1769), (698, 1769), (699, 1772), (700, 1773), (701, 1774), (702, 1775), (703, 1778), (704, 1779), (705, 1780), (706, 1781), (707, 1782), (708, 1783), (709, 1784), (710, 1785), (711, 1784), (712, 1787), (713, 1788), (715, 1794), (716, 1795), (717, 1801), (718, 1802), (719, 1803), (720, 1804), (721, 1805), (722, 1805), (724, 1808), (725, 1810),

In [6]:
def find_continuous_frames(frame_numbers):
    continuous_frames = []
    start = frame_numbers[0]
    end = frame_numbers[0]

    for i in range(1, len(frame_numbers)):
        if frame_numbers[i] == end + 1:
            end = frame_numbers[i]
        else:
            continuous_frames.append((start, end))
            start = frame_numbers[i]
            end = frame_numbers[i]

    continuous_frames.append((start, end))
    return continuous_frames

def create_video_from_continuous_frames(input_video, continuous_frames, output_video_path):
        clips = []
        for start, end in continuous_frames:

            if start == end:
                end += 1

            print(start, end)
            
            clip = input_video.subclip(start, end)
            audio_clip = input_video.subclip(start, end).audio

            clip.set_audio(audio_clip).write_videofile(output_video_path, codec="libx264", audio_codec="aac")
            
            clips.append(clip)
        
        final_clip = mp.concatenate_videoclips(clips)
        final_clip.write_videofile(output_video_path, codec="libx264", audio_codec="aac")

        final_clip.close()        




# 사용 예시
continous_frames = find_continuous_frames([pair[0] for pair in overlapping_frames])
print(continous_frames)
output_video_path = "/home/taehoonlee/KT/mp4s/diggled/S01E03.mp4"
create_video_from_continuous_frames(merged_video, continous_frames, output_video_path)    

[(0, 0), (23, 23), (99, 99), (124, 124), (346, 347), (350, 351), (365, 373), (409, 411), (415, 415), (431, 433), (628, 631), (633, 636), (683, 713), (715, 722), (724, 751), (809, 809), (847, 847), (875, 876), (946, 965), (967, 977), (979, 981), (983, 989), (991, 998), (1026, 1043), (1045, 1052), (1054, 1074), (1077, 1087), (1089, 1090), (1092, 1114), (1116, 1160), (1162, 1223), (1225, 1303), (1305, 1335), (1412, 1412), (1509, 1510), (1529, 1555), (1557, 1558), (1560, 1560), (1564, 1566), (1568, 1568)]
0 1
Moviepy - Building video /home/taehoonlee/KT/mp4s/diggled/SO1EO3.mp4.
MoviePy - Writing audio in SO1EO3TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /home/taehoonlee/KT/mp4s/diggled/SO1EO3.mp4



Moviepy - Done !
Moviepy - video ready /home/taehoonlee/KT/mp4s/diggled/SO1EO3.mp4
23 24
Moviepy - Building video /home/taehoonlee/KT/mp4s/diggled/SO1EO3.mp4.
MoviePy - Writing audio in SO1EO3TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /home/taehoonlee/KT/mp4s/diggled/SO1EO3.mp4



Moviepy - Done !
Moviepy - video ready /home/taehoonlee/KT/mp4s/diggled/SO1EO3.mp4
99 100
Moviepy - Building video /home/taehoonlee/KT/mp4s/diggled/SO1EO3.mp4.
MoviePy - Writing audio in SO1EO3TEMP_MPY_wvf_snd.mp4


...

Moviepy - Done !
Moviepy - video ready /home/taehoonlee/KT/mp4s/diggled/SO1EO3.mp4
1568 1569
Moviepy - Building video /home/taehoonlee/KT/mp4s/diggled/SO1EO3.mp4.
MoviePy - Writing audio in SO1EO3TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /home/taehoonlee/KT/mp4s/diggled/SO1EO3.mp4



Moviepy - Done !
Moviepy - video ready /home/taehoonlee/KT/mp4s/diggled/SO1EO3.mp4
Moviepy - Building video /home/taehoonlee/KT/mp4s/diggled/SO1EO3.mp4.
MoviePy - Writing audio in SO1EO3TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /home/taehoonlee/KT/mp4s/diggled/SO1EO3.mp4



Moviepy - Done !
Moviepy - video ready /home/taehoonlee/KT/mp4s/diggled/SO1EO3.mp4
